In [1]:
from StarWars.data import get_data

In [2]:
get_data('train', 10)

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.000000,...,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.000000,0.325512,100008.jpg
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.000000,...,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.000000,0.000000,100023.jpg
2,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100053.jpg
3,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.000000,...,0.094549,0.000000,0.094549,0.189098,0.000000,0.000000,0.000000,0.000000,0.000000,100078.jpg
4,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100090.jpg
5,0.738832,0.238159,0.023009,0.000000,0.238159,0.000000,0.238159,0.000000,0.238159,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100122.jpg
6,0.462492,0.456033,0.081475,0.000000,0.456033,0.000000,0.456033,0.000000,0.456033,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100123.jpg
7,0.687783,0.288344,0.023873,0.000000,0.288344,0.069098,0.219246,0.000000,0.288344,0.067228,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100128.jpg
8,0.021834,0.976952,0.001214,0.021751,0.955201,0.313077,0.642124,0.546491,0.408711,0.160096,...,0.207253,0.152044,0.187194,0.000000,0.000000,0.054649,0.081974,0.081974,0.327894,100134.jpg
9,0.269843,0.730157,0.000000,0.730157,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100143.jpg


In [3]:
import pandas as pd
from google.cloud import storage
import gcsfs
import numpy as np
from PIL import Image
import pandas as pd

In [4]:
BUCKET_DATA_PATH = "gs://lw-verspieren-starwars/data/"


def rename(GalaxyID):
    return str(GalaxyID)+'.jpg'

def get_data(y, nrows): # add nrows to sample
    '''returns a DataFrame with nrows from s3 bucket'''

    if y == 'train':
        # Loading DataFrames
        df_y = pd.read_csv(f'{BUCKET_DATA_PATH}training_solutions_rev1.csv', nrows=nrows)
        df_images =  pd.read_csv(f'{BUCKET_DATA_PATH}images_liste_train.csv', nrows=nrows)

        # Checking images corresponds to df
        list_images = list(df_images['0'])
        df_Galaxy_ID = pd.DataFrame([list_images[i].replace('.jpg','') for i in range(len(list_images))]).rename(columns={0:'GalaxyID'}).astype('int64')
        df = df_y.merge(df_Galaxy_ID, on='GalaxyID', how='inner')

        # Preparing df
        df['image'] = df_images['0']
        df_data = df.drop(columns=['GalaxyID'])

        return df_data

    if y == 'test':
        # Loading DataFrame
        df_y = pd.read_csv(f'{BUCKET_DATA_PATH}central_pixel_benchmark.csv', nrows=nrows)

        # Preparing df
        df_y['image'] = df_y['GalaxyID'].apply(rename)
        df_data = df_y.drop(columns=['GalaxyID'])

        return df_data
    return print("Please provide 'train' or 'test' ")


In [5]:
get_data('train', 10)

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.000000,...,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.000000,0.325512,100008.jpg
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.000000,...,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.000000,0.000000,100023.jpg
2,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100053.jpg
3,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.000000,...,0.094549,0.000000,0.094549,0.189098,0.000000,0.000000,0.000000,0.000000,0.000000,100078.jpg
4,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100090.jpg
5,0.738832,0.238159,0.023009,0.000000,0.238159,0.000000,0.238159,0.000000,0.238159,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100122.jpg
6,0.462492,0.456033,0.081475,0.000000,0.456033,0.000000,0.456033,0.000000,0.456033,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100123.jpg
7,0.687783,0.288344,0.023873,0.000000,0.288344,0.069098,0.219246,0.000000,0.288344,0.067228,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100128.jpg
8,0.021834,0.976952,0.001214,0.021751,0.955201,0.313077,0.642124,0.546491,0.408711,0.160096,...,0.207253,0.152044,0.187194,0.000000,0.000000,0.054649,0.081974,0.081974,0.327894,100134.jpg
9,0.269843,0.730157,0.000000,0.730157,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100143.jpg


In [6]:
def load_image(image):
    '''returns one image'''
    folder = f'{BUCKET_DATA_PATH}images_train/{image}'
    img = Image.open(folder)
    img_array = np.array(img)
    return np.resize(img_array, (224,224,3))

In [7]:
load_image('100008.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'gs://lw-verspieren-starwars/data/images_train/100008.jpg'

In [ ]:
lw-verspieren-starwars/data/images_train/100008.jpg

In [9]:
pd.read_csv(f'gs://lw-verspieren-starwars/data/training_solutions_rev1.csv', nrows=10)

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.000000,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.000000,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.000000,0.000000,0.000000,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,100122,0.738832,0.238159,0.023009,0.000000,0.238159,0.000000,0.238159,0.000000,0.238159,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,100123,0.462492,0.456033,0.081475,0.000000,0.456033,0.000000,0.456033,0.000000,0.456033,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,100128,0.687783,0.288344,0.023873,0.000000,0.288344,0.069098,0.219246,0.000000,0.288344,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,100134,0.021834,0.976952,0.001214,0.021751,0.955201,0.313077,0.642124,0.546491,0.408711,...,0.021751,0.207253,0.152044,0.187194,0.000000,0.000000,0.054649,0.081974,0.081974,0.327894
9,100143,0.269843,0.730157,0.000000,0.730157,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.168728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
Image.open(f'gs://lw-verspieren-starwars/data/images_train/100008.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'gs://lw-verspieren-starwars/data/images_train/100008.jpg'

In [16]:
from google.colab import drive
drive.mount('/content/drive')
fs = gcsfs.GCSFileSystem(project='wagon-bootcamp-305110',token='drive/MyDrive/gcp_keys/star-wars-key.json')

ModuleNotFoundError: No module named 'google.colab'

In [15]:
!pip install google.colab

     |████████████████████████████████| 72 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 64 kB 4.8 MB/s eta 0:00:011
     |████████████████████████████████| 104 kB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 758 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.8 MB 64 kB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 6.5 MB/s eta 0:00:011
     |████████████████████████████████| 484 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 245 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 6.5 MB/s eta 0:00:01


  Created wheel for google.colab: filename=google_colab-1.0.0-py2.py3-none-any.whl size=102289 sha256=155e195e0a0af2f557ed0d58a52235931d72150bd7cbbc9957e80a57009e5366
  Stored in directory: /Users/mickaelkomendyak/Library/Caches/pip/wheels/e4/e7/91/b2736701bca00e273fdc79f80e2727b558e0903d81b758eb69
anceled
ERROR: Operation cancelled by user
